In [ ]:
import pandas as pd

games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")
ranking = pd.read_csv("Data/ranking.csv")
player = pd.read_csv("Data/players.csv")

**Golden State Warriors우승 시즌과 이전 시즌의 스타팅 라인업 및 주요 선수 활용 변화 분석(감독관점)**

Title : 우승 시즌에 어떤 선수를 어떻게 썼길래 결과가 달라졌을까?
     우승 시즌에 고정적으로 기용된 스타팅 멤버는 누구?
     이전 시즌 대비 출전 시간이 늘어난 선수는?
     팀 성과 개선이 특정 선수때문일까? 라인업/운영 변화일까?

“우승 시즌과 이전 시즌의 스타팅 라인업 및 주요 선수 활용 변화를 비교하여,
감독의 운영 변화가 팀 성과에 미친 영향을 분석하고 향후 선수 영입 전략에 활용한다.”

전처리 계획
     1. 우승시즌과 직전 전체 시즌 비교
     2. 스타팅 라인업 변수를 만들어서 시즌 전체 경기 선발 비율확인


사용할 컬럼 (games_details, games, teams)
    1. 우승시즌비교
         SEASON
         GAME_ID
         TEAM_ID
    2. 스타팅 라인업 구성
         GAME_ID
         PLAYER_NAME
         START_POSITION
         SEASON

In [ ]:
games.info()

In [ ]:
details.info()

In [ ]:
teams.info()

In [ ]:
games.head(1)

In [ ]:
details.head(1)

In [ ]:
# 팀 약어 리스트
teams['ABBREVIATION'].unique()
# GSW

In [ ]:
#TEAM_ID
teams[teams['ABBREVIATION'] == 'GSW']

# TEAM_ID = 1610612744

In [ ]:
#사용할 컬럼 NULL값 확인하기
games[['GAME_ID', 'SEASON', 'HOME_TEAM_WINS']].isna().sum()

In [ ]:
#GAME_ID 중복 여부 확인 -> 0
games['GAME_ID'].duplicated().sum

In [ ]:
#games테이블에 SEASON 유니크값들
games['SEASON'].unique()

In [ ]:
#GSW가 참여한 경기 모두(홈경기 + 원정경기) => games_gsw 데이터프레임만듬
GSW_ID = 1610612744

games_gsw = games[(games['HOME_TEAM_ID'] == GSW_ID) |
                  (games['VISITOR_TEAM_ID'] == GSW_ID)]

games_gsw

In [ ]:
#GSW의 일부 필요 컬럼을 => games_gsw_part 데이터 프레임으로 만듬
games_gsw_part = games_gsw[['GAME_ID', 'SEASON', 'HOME_TEAM_WINS']]


In [ ]:
#details테이블에 위에만든 games_gsw_part 데이터 프레임의 GAME_ID 기준으로 INNER JOIN 진행
merged_gsw = details.merge(games_gsw_part, on='GAME_ID', how='inner')
merged_gsw

In [ ]:
#GSW만 필터링작업
gsw = merged_gsw = merged_gsw[merged_gsw['TEAM_ABBREVIATION'] == 'GSW']
gsw.head()

In [ ]:
#우승 직전/후로 나뉘니까 2016부터는 데이터 불필요.
gsw = gsw[gsw['SEASON'] <= 2015]

In [ ]:
#우승시즌 2014-2015, 우승직전시즌 전체
#champioship 2014-2015 우승시즌
#pre-championship 2013-2014 우승바로 직전 시즌이 직전 시즌
def season_group(season):
    if season == 2015:
        return "champioship(2014-2015)" 
    elif season == 2014:
        return "pre-championship(2013-2014)"
    
gsw['SEASON_GROUP'] = gsw['SEASON'].apply(season_group)

In [ ]:
#스타팅 라인업 -> START_POSITION 값이 있는 것만
starters = gsw[gsw['START_POSITION'].notna()]
starters

In [ ]:
#시즌별 스타팅 횟수 집계
starter_count = (starters.groupby(['SEASON_GROUP', 'PLAYER_NAME']).size().reset_index(name='start_games'))
starter_count.head(10)

In [ ]:
# 각 시즌별 스타팅 횟수가 많은 순서대로 나열.
starter_count.sort_values(['SEASON_GROUP', 'start_games'], ascending=False).head(10)

In [ ]:
# 각시즌 5명만 추리기
top_starters = (
    starter_count
    .sort_values(['SEASON_GROUP', 'start_games'], ascending=False)
    .groupby('SEASON_GROUP')
    .head(5)
)
top_starters

In [ ]:
!pip install matplotlib


In [ ]:
#pivot
plot_df = (
    top_starters
    .pivot(
        index='PLAYER_NAME',
        columns='SEASON_GROUP',
        values='start_games'
    )
)
plot_df

In [ ]:
#막대그래프 시각화
import matplotlib.pyplot as plt

plot_df.plot(
    kind='bar',
    figsize=(10, 5)
)

plt.title("GSW Starting Lineup : Before vs Championship Season")
plt.xlabel("Player")
plt.ylabel("Number of Starts")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


막대그래프를 보았을 때 
- Curry, Thompson 이전시즌부터 거의 풀타임 선발
- Green 우승시즌에 선발 급증
- harrison 선발 횟수 줄어듬

**선수별 평균 출전시간(MIN)**
스타팅횟수와 출전시간이 늘어났다면 그만큼 실력이 있어서 감독의 신뢰가 있는 선수가 아닐까?

In [ ]:
#MIN(출전시간) 데이터 셋 확인
# MIN                      str
details.dtypes

In [ ]:
#문자열 --> 숫자로 변환 할 필요 있음.
def min_to_float(x):
    if pd.isna(x):
        return 0

    x = str(x)

    if ':' in x:
        m, s = x.split(':')
        return int(m) + int(s) / 60

    if x.isdigit():
        return int(x)

    return 0
gsw['MIN_FLOAT'] = gsw['MIN'].apply(min_to_float)

In [ ]:
gsw['MIN_FLOAT']